In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableSequence
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import os


In [3]:
#Read OPENAI API
with open('keys/api_openai.txt', 'r') as file:
    api_openai = file.read()
os.environ['OPENAI_API_KEY'] = api_openai

llm = ChatOpenAI(model= "gpt-4.1-2025-04-14", temperature=0)

In [20]:
# 1. Función que convierte el JSON en un texto descriptivo
def procesar_metricas(data):
    texto = []
    for rep, fases in data.items():
        texto.append(f"Análisis para la repetición {rep}:\n")
        for fase, angulos in fases.items():
            texto.append(f"  Fase {fase}:\n")
            for articulacion, valores in angulos.items():
                min_val = valores["min"]
                max_val = valores["max"]
                texto.append(f"    - {articulacion}: min = {min_val}°, max = {max_val}°\n")
    return {"analisis_metricas": "\n".join(texto)}

In [27]:
#read rules
with open('keys/context.txt') as file:
    rules = file.read()

#prompt
compare_metrics = ChatPromptTemplate.from_messages([
    ("system", rules),
    ("user", "Da recomendaciones, sugerencias, comentarios sobre las metricas obtenidas del usuario:\n{{analisis_metricas}}")
])

In [28]:
# 5. Input de prueba
input_data = {
    "rep1": {
        "fase1": {
            "angulo_rodilla": {"min": 130, "max": 160},
            "angulo_cadera": {"min": 140, "max": 170},
            "angulo_espalda": {"min": 100, "max": 130}
        },
        "fase2": {
            "angulo_rodilla": {"min": 90, "max": 130},
            "angulo_cadera": {"min": 100, "max": 140},
            "angulo_espalda": {"min": 95, "max": 135}
        },
        "fase3": {
            "angulo_rodilla": {"min": 45, "max": 70},
            "angulo_cadera": {"min": 80, "max": 100},
            "angulo_espalda": {"min": 60, "max": 90}
        },
        "fase4": {
            "angulo_rodilla": {"min": 125, "max": 160},
            "angulo_cadera": {"min": 135, "max": 165},
            "angulo_espalda": {"min": 110, "max": 145}
        }
    }
}

In [29]:
# 4. Cadena LangChain
chain = (
    RunnableLambda(procesar_metricas) |
    compare_metrics |
    llm |
    StrOutputParser()
)

In [31]:
# 6. Ejecutar la cadena
respuesta = chain.invoke(procesar_metricas(input_data))
print(respuesta)

AttributeError: 'str' object has no attribute 'items'

In [25]:
val = procesar_metricas(input_data)

In [26]:
val

{'analisis_metricas': 'Análisis para la repetición rep1:\n\n  Fase fase1:\n\n    - angulo_rodilla: min = 130°, max = 160°\n\n    - angulo_cadera: min = 140°, max = 170°\n\n    - angulo_espalda: min = 100°, max = 130°\n\n  Fase fase2:\n\n    - angulo_rodilla: min = 90°, max = 130°\n\n    - angulo_cadera: min = 100°, max = 140°\n\n    - angulo_espalda: min = 95°, max = 135°\n\n  Fase fase3:\n\n    - angulo_rodilla: min = 45°, max = 70°\n\n    - angulo_cadera: min = 80°, max = 100°\n\n    - angulo_espalda: min = 60°, max = 90°\n\n  Fase fase4:\n\n    - angulo_rodilla: min = 125°, max = 160°\n\n    - angulo_cadera: min = 135°, max = 165°\n\n    - angulo_espalda: min = 110°, max = 145°\n'}

In [32]:
from typing import Dict

In [33]:
def convert_squat_data_to_text(input_data: Dict) -> str:
    """
    Convierte los datos JSON de ángulos de sentadilla a texto plano legible
    """
    text_output = []
    
    for rep_key, rep_data in input_data.items():
        text_output.append(f"=== ANÁLISIS DE {rep_key.upper()} ===\n")
        
        for fase_key, fase_data in rep_data.items():
            text_output.append(f"🔸 {fase_key.upper()}:")
            
            for angulo, valores in fase_data.items():
                angulo_nombre = angulo.replace('_', ' ').title()
                text_output.append(f"  • {angulo_nombre}: {valores['min']}° - {valores['max']}°")
            
            text_output.append("")  # Línea vacía entre fases
        
        text_output.append("-" * 50 + "\n")
    
    return "\n".join(text_output)

In [ ]:
def create_squat_analysis_chain():
    """
    Crea la cadena de LangChain para análisis de sentadillas
    """
    
    # Conocimiento base sobre técnica de sentadillas
    squat_knowledge = rules
    
    # Template del prompt
    prompt_template = PromptTemplate(
        input_variables=["squat_knowledge", "measurement_data"],
        template="""
        Eres un entrenador personal experto en biomecánica del ejercicio. Analiza los siguientes datos de ángulos corporales durante la ejecución de sentadillas y proporciona sugerencias específicas para mejorar la técnica.

        {squat_knowledge}

        DATOS DE MEDICIÓN:
        {measurement_data}

        INSTRUCCIONES:
        1. Compara los ángulos medidos con los rangos ideales
        2. Identifica desviaciones significativas (>10° fuera del rango ideal)
        3. Proporciona sugerencias específicas y accionables
        4. Prioriza las correcciones más importantes para la seguridad
        5. Usa un lenguaje claro y motivador

        FORMATO DE RESPUESTA:
        📊 ANÁLISIS BIOMECÁNICO:
        [Resumen general del rendimiento]

        ⚠️ ÁREAS DE MEJORA:
        [Lista de problemas identificados por fase]

        💡 SUGERENCIAS ESPECÍFICAS:
        [Recomendaciones concretas para cada problema]

        🎯 EJERCICIOS RECOMENDADOS:
        [Ejercicios auxiliares para mejorar la técnica]

        Por favor, proporciona un análisis detallado y constructivo:
        """
    )
    
    # Configurar el LLM (ajusta según tu configuración)
    llm = OpenAI(
        temperature=0.3,  # Baja temperatura para respuestas más consistentes
        max_tokens=1000
    )
    
    # Crear la cadena
    analysis_chain = LLMChain(
        llm=llm,
        prompt=prompt_template,
        output_parser=SquatAnalysisOutputParser(),
        verbose=True
    )
    
    return analysis_chain, squat_knowledge

def analyze_squat_performance(input_data: Dict) -> Dict[str, Any]:
    """
    Función principal que ejecuta todo el pipeline de análisis
    """
    
    # Paso 1: Convertir JSON a texto plano
    text_data = convert_squat_data_to_text(input_data)
    
    # Paso 2: Crear la cadena de análisis
    analysis_chain, squat_knowledge = create_squat_analysis_chain()
    
    # Paso 3: Ejecutar el análisis
    result = analysis_chain.run({
        "squat_knowledge": squat_knowledge,
        "measurement_data": text_data
    })
    
    return result